In [1]:
import xgboost as xgb
from sklearn.mixture import GaussianMixture

## load trained algorithms

### load SAVIC-P - CB XGB classifier 

In [2]:
xgbc_sus_cb = xgb.XGBClassifier()
xgbc_sus_cb.load_model('Output/ML/models/xgbc_sus_cb.json')

### load SAVIC-Q - CB XGB classifier \& regressor

In [3]:
xgbc_kcb = xgb.XGBClassifier()
xgbc_kcb.load_model('Output/ML/models/xgbc_kcb.json')

xgbr_c1_b1 = xgb.XGBRegressor()
xgbr_c1_b1.load_model('Output/ML/models/xgbr_cb_c1_b1.json')

xgbr_c1_b0 = xgb.XGBRegressor()
xgbr_c1_b0.load_model('Output/ML/models/xgbr_cb_c1_b0.json')

xgbr_c0_b1_k0 = xgb.XGBRegressor()
xgbr_c0_b1_k0.load_model('Output/ML/models/xgbr_cb_c0_b1_k0.json')

xgbr_c0_b1_k1 = xgb.XGBRegressor()
xgbr_c0_b1_k1.load_model('Output/ML/models/xgbr_cb_c0_b1_k1.json')

### load SAVIC-C - CB GM Classifier

In [4]:
gmm_cb_name = 'Output/ML/models/GMM_CB/GMM_CB'
means = np.load(gmm_cb_name + '_means.npy')
covar = np.load(gmm_cb_name + '_covariances.npy')
gmm_cb = GaussianMixture(n_components = len(means), covariance_type='full')
gmm_cb.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
gmm_cb.weights_ = np.load(gmm_cb_name + '_weights.npy')
gmm_cb.means_ = means
gmm_cb.covariances_ = covar


## SAVIC-P - CB

### load SAVIC-P - CB processing function

In [5]:
def SAVIC_P_CB(df_cb_):
    df_cb_pre_SP_ = df_cb_.copy()
    df_cb_pre_SP_[['beta_par_core', 'alph_c', 'tau_b', 'alph_b']] = \
    np.log10(df_cb_pre_SP_[['beta_par_core', 'alph_c', 'tau_b', 'alph_b']])
    df_cb_pre_SP_['unstable'] = xgbc_sus_cb.predict(df_cb_pre_SP_.values).astype('bool')
    
    df_cb_post_SP_ = df_cb_pre_SP_.copy()
    df_cb_post_SP_[['beta_par_core', 'alph_c', 'tau_b', 'alph_b']] = \
    pow(10., df_cb_post_SP_[['beta_par_core', 'alph_c', 'tau_b', 'alph_b']])
    
    return df_cb_post_SP_

### run SAVIC-P - CB 

In [6]:
df_cb_post_SP = SAVIC_P_CB(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-P_CB_pre_P'))
df_cb_post_SP

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,unstable
0,1.0,1.0,1.0,1.0,0.05,0.5,False
1,1.5,2.5,0.8,1.0,0.05,0.5,True
2,0.5,1.0,1.0,3.5,0.10,1.5,True
3,0.8,1.1,1.0,1.2,0.05,1.8,True
4,0.5,0.7,0.8,0.8,0.01,0.2,False
5,0.8,3.1,1.0,3.9,0.10,1.9,True


## SAVIC-Q - CB

### load SAVIC-Q - CB processing function

In [7]:
def SAVIC_Q_CB(df_cb_):
    
    df_cb_uns_ = df_cb_[df_cb_['unstable']].drop(columns = ['unstable'])
    df_cb_uns_pre_SQ_ = df_cb_uns_.copy()
    df_cb_uns_pre_SQ_[['beta_par_core', 'alph_c', 'alph_b']] = \
    np.log10(df_cb_uns_pre_SQ_[['beta_par_core', 'alph_c', 'alph_b']])
    df_cb_uns_pre_SQ_.columns = [['log_beta_par_core', 'log_alph_c', 'tau_b', 'log_alph_b', 'D_b', 'vv_b']]
    
    df_cb_uns_postC_SQ_ = df_cb_uns_pre_SQ_.copy()
    df_cb_uns_postC_SQ_['group'] = xgbc_kcb.predict(df_cb_uns_pre_SQ_.values)
    
    df_cb_uns_preQ_SQ_ = df_cb_uns_postC_SQ_.copy()
    df_cb_uns_preQ_SQ_[['log_beta_par_core', 'log_alph_c', 'log_alph_b']] = \
    pow(10., df_cb_uns_preQ_SQ_[['log_beta_par_core', 'log_alph_c', 'log_alph_b']]) 
    df_cb_uns_preQ_SQ_.columns = ['beta_par_core', 'alph_c', 'tau_b', 'alph_b', 'D_b', 'vv_b', 'group']
    
    # regressors
    df_cb_uns_preQ_SQ_0_ = df_cb_uns_preQ_SQ_[df_cb_uns_preQ_SQ_['group'] == 0].drop(columns = ['group'])
    df_cb_uns_preQ_SQ_1_ = df_cb_uns_preQ_SQ_[df_cb_uns_preQ_SQ_['group'] == 1].drop(columns = ['group'])
    df_cb_uns_preQ_SQ_2_ = df_cb_uns_preQ_SQ_[df_cb_uns_preQ_SQ_['group'] == 2].drop(columns = ['group'])
    df_cb_uns_preQ_SQ_3_ = df_cb_uns_preQ_SQ_[df_cb_uns_preQ_SQ_['group'] == 3].drop(columns = ['group'])
    df_cb_uns_preQ_SQ_4_ = df_cb_uns_preQ_SQ_[df_cb_uns_preQ_SQ_['group'] == 4].drop(columns = ['group'])
    df_cb_uns_preQ_SQ_5_ = df_cb_uns_preQ_SQ_[df_cb_uns_preQ_SQ_['group'] == 5].drop(columns = ['group'])
    
    
    if df_cb_uns_preQ_SQ_0_.size > 0:
        df_cb_uns_preQ_SQ_0_log_ = df_cb_uns_preQ_SQ_0_.copy()

        df_cb_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_b']] = \
        np.log10(df_cb_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_b']])
        
        df_cb_uns_preQ_SQ_0_log_[['Pow_core', 'Pow_beam', 'kB_angle']] = \
        xgbr_c1_b1.predict(df_cb_uns_preQ_SQ_0_log_).astype(float)

        df_cb_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'Pow_beam', 'kB_angle']] = \
        10**df_cb_uns_preQ_SQ_0_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'Pow_beam', 'kB_angle']]

        df_cb_uns_postQ_SQ_0_ = df_cb_uns_preQ_SQ_0_log_.copy()
        df_cb_uns_postQ_SQ_0_ = df_cb_uns_postQ_SQ_0_[['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle']]
    else:
        df_cb_uns_postQ_SQ_0_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle'])

    if df_cb_uns_preQ_SQ_1_.size > 0:
        df_cb_uns_preQ_SQ_1_log_ = df_cb_uns_preQ_SQ_1_.copy()


        df_cb_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_b']] = \
        np.log10(df_cb_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_b']])
        
        df_cb_uns_preQ_SQ_1_log_[['Pow_core', 'Pow_beam', 'kB_angle']] = \
        xgbr_c1_b1.predict(df_cb_uns_preQ_SQ_1_log_).astype(float)

        df_cb_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'Pow_beam', 'kB_angle']] = \
        10**df_cb_uns_preQ_SQ_1_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'Pow_beam', 'kB_angle']]

        df_cb_uns_postQ_SQ_1_ = df_cb_uns_preQ_SQ_1_log_.copy()
        df_cb_uns_postQ_SQ_1_ = df_cb_uns_postQ_SQ_1_[['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle']]
    else:
        df_cb_uns_postQ_SQ_1 = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle'])

    if df_cb_uns_preQ_SQ_2_.size > 0:
        df_cb_uns_preQ_SQ_2_log_ = df_cb_uns_preQ_SQ_2_.copy()


        df_cb_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_b']] = \
        np.log10(df_cb_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_b']])
        
        df_cb_uns_preQ_SQ_2_log_[['Pow_core', 'kB_angle']] = \
        xgbr_c1_b0.predict(df_cb_uns_preQ_SQ_2_log_).astype(float)

        df_cb_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'kB_angle']] = \
        10**df_cb_uns_preQ_SQ_2_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'kB_angle']]

        df_cb_uns_postQ_SQ_2_ = df_cb_uns_preQ_SQ_2_log_.copy()
        df_cb_uns_postQ_SQ_2_['Pow_beam'] = 0
        df_cb_uns_postQ_SQ_2_ = df_cb_uns_postQ_SQ_2_[['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle']]
    else:
        df_cb_uns_postQ_SQ_2_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle'])

    if df_cb_uns_preQ_SQ_3_.size > 0:
        df_cb_uns_preQ_SQ_3_log_ = df_cb_uns_preQ_SQ_3_.copy()


        df_cb_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_b']] = \
        np.log10(df_cb_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_b']])
        
        df_cb_uns_preQ_SQ_3_log_[['Pow_core', 'kB_angle']] = \
        xgbr_c1_b0.predict(df_cb_uns_preQ_SQ_3_log_).astype(float)

        df_cb_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'kB_angle']] = \
        10**df_cb_uns_preQ_SQ_3_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_core', 'kB_angle']]

        df_cb_uns_postQ_SQ_3_ = df_cb_uns_preQ_SQ_3_log_.copy()
        df_cb_uns_postQ_SQ_3_['Pow_beam'] = 0
        df_cb_uns_postQ_SQ_3_ = df_cb_uns_postQ_SQ_3_[['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle']]
    else:
        df_cb_uns_postQ_SQ_3_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle'])

    if df_cb_uns_preQ_SQ_4_.size > 0:
        df_cb_uns_preQ_SQ_4_log_ = df_cb_uns_preQ_SQ_4_.copy()


        df_cb_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_b']] = \
        np.log10(df_cb_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_b']])
        
        df_cb_uns_preQ_SQ_4_log_[['Pow_beam', 'kB_angle']] = \
        xgbr_c0_b1_k1.predict(df_cb_uns_preQ_SQ_4_log_).astype(float)

        df_cb_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_beam', 'kB_angle']] = \
        10**df_cb_uns_preQ_SQ_4_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_beam', 'kB_angle']]

        df_cb_uns_postQ_SQ_4_ = df_cb_uns_preQ_SQ_4_log_.copy()
        df_cb_uns_postQ_SQ_4_['Pow_core'] = 0
        df_cb_uns_postQ_SQ_4_ = df_cb_uns_postQ_SQ_4_[['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle']]
    else:
        df_cb_uns_postQ_SQ_4_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle'])

    if df_cb_uns_preQ_SQ_5_.size > 0:
        df_cb_uns_preQ_SQ_5_log_ = df_cb_uns_preQ_SQ_5_.copy()


        df_cb_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_b']] = \
        np.log10(df_cb_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_b']])
        
        df_cb_uns_preQ_SQ_5_log_[['Pow_beam', 'kB_angle']] = \
        xgbr_c0_b1_k0.predict(df_cb_uns_preQ_SQ_5_log_).astype(float)

        df_cb_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_beam', 'kB_angle']] = \
        10**df_cb_uns_preQ_SQ_5_log_[['beta_par_core', 'alph_c','alph_b', 'Pow_beam', 'kB_angle']]

        df_cb_uns_postQ_SQ_5_ = df_cb_uns_preQ_SQ_5_log_.copy()
        df_cb_uns_postQ_SQ_5_['Pow_core'] = 0
        df_cb_uns_postQ_SQ_5_ = df_cb_uns_postQ_SQ_5_[['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle']]
    else:
        df_cb_uns_postQ_SQ_5_ = pd.DataFrame(columns = ['beta_par_core', 'alph_c', 'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', 'kB_angle'])

    df_cb_uns_postQ_SQ_ = pd.concat([df_cb_uns_postQ_SQ_0_, df_cb_uns_postQ_SQ_1_, df_cb_uns_postQ_SQ_2_, \
                                 df_cb_uns_postQ_SQ_3_, df_cb_uns_postQ_SQ_4_, df_cb_uns_postQ_SQ_5_]).sort_index()
    
    df_cb_postQ_SQ_ = df_cb_.copy()

    df_cb_postQ_SQ_['group'] = np.nan
    df_cb_postQ_SQ_.loc[df_cb_uns_preQ_SQ_.index, 'group' ] = \
    df_cb_uns_preQ_SQ_['group']
    df_cb_postQ_SQ_['group'] = df_cb_postQ_SQ_['group']#.astype('Int64')

    df_cb_postQ_SQ_[['Pow_core', 'Pow_beam', 'kB_angle']] = np.nan
    df_cb_postQ_SQ_.loc[df_cb_uns_postQ_SQ_.index, ['Pow_core', 'Pow_beam', 'kB_angle'] ] = \
    df_cb_uns_postQ_SQ_[['Pow_core', 'Pow_beam', 'kB_angle'] ]
    
    return df_cb_postQ_SQ_

### run SAVIC-Q - CB 

In [8]:
df_cb_post_SQ = SAVIC_Q_CB(df_cb_post_SP)
df_cb_post_SQ

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,unstable,group,Pow_core,Pow_beam,kB_angle
0,1.0,1.0,1.0,1.0,0.05,0.5,False,NaN,NaN,NaN,NaN
1,1.5,2.5,0.8,1.0,0.05,0.5,True,3.0,0.193271,0.000000,0.004137
2,0.5,1.0,1.0,3.5,0.10,1.5,True,5.0,0.000000,0.123028,0.003983
3,0.8,1.1,1.0,1.2,0.05,1.8,True,5.0,0.000000,0.004923,0.001414
4,0.5,0.7,0.8,0.8,0.01,0.2,False,NaN,NaN,NaN,NaN
5,0.8,3.1,1.0,3.9,0.10,1.9,True,1.0,0.214777,0.000728,0.000991


## SAVIC-C - CB

### load SAVIC-C - CB processing function

In [9]:
def SAVIC_C_CB(df_cb_):
    
    df_cb_uns_ = df_cb_[df_cb_['unstable']].drop(columns = ['unstable'])
    df_cb_uns_pre_SC_ = df_cb_uns_.copy()

    df_cb_uns_pre_SC_['uns_IC'] = (1. + 0.367 / ( df_cb_uns_pre_SC_['beta_par_core'] - 0.011 )**0.364) < df_cb_uns_pre_SC_['alph_c']
    df_cb_uns_pre_SC_['uns_Mirror'] = (1. + 0.702 / ( df_cb_uns_pre_SC_['beta_par_core'] + 0.009 )**0.674) < df_cb_uns_pre_SC_['alph_c']
    df_cb_uns_pre_SC_['uns_FM'] = (1. - 0.408 / ( df_cb_uns_pre_SC_['beta_par_core'] - 0.410 )**0.529) > df_cb_uns_pre_SC_['alph_c']
    df_cb_uns_pre_SC_['uns_OFH'] = (1. - 1.454 / ( df_cb_uns_pre_SC_['beta_par_core'] + 0.178 )**1.023) > df_cb_uns_pre_SC_['alph_c']
    df_cb_uns_pre_SC_['uns_core'] = df_cb_uns_pre_SC_['uns_OFH'] | df_cb_uns_pre_SC_['uns_FM'] | df_cb_uns_pre_SC_['uns_Mirror'] | df_cb_uns_pre_SC_['uns_IC']

    df_cb_uns_pre_SC_['beta_par_beam'] = df_cb_uns_pre_SC_['beta_par_core'] * df_cb_uns_pre_SC_['D_b'] / df_cb_uns_pre_SC_['tau_b']
    df_cb_uns_pre_SC_['lambda_FH_beta'] = ( (df_cb_uns_pre_SC_['beta_par_core'] + df_cb_uns_pre_SC_['beta_par_beam']) - \
    (df_cb_uns_pre_SC_['beta_par_core'] * df_cb_uns_pre_SC_['alph_c'] + df_cb_uns_pre_SC_['beta_par_beam'] * df_cb_uns_pre_SC_['alph_b']) ) / 2.
    df_cb_uns_pre_SC_['lambda_FH_beam'] = df_cb_uns_pre_SC_['D_b'] * df_cb_uns_pre_SC_['vv_b']**2.
    df_cb_uns_pre_SC_['lambda_FH'] = df_cb_uns_pre_SC_['lambda_FH_beta'] + df_cb_uns_pre_SC_['lambda_FH_beam']# + df_cb_uns_pre_SC_['lambda_FH_alpha']
    df_cb_uns_pre_SC_['lambda_FH'] = df_cb_uns_pre_SC_['lambda_FH'] > 1.

    df_cb_uns_pre_SC_['Pow_core'][df_cb_uns_pre_SC_['Pow_core'] < 0] = 0
    df_cb_uns_pre_SC_['Pow_beam'][df_cb_uns_pre_SC_['Pow_beam'] < 0] = 0

    
    column_list_cb_ = ['beta_par_core', 'alph_c', 'kB_angle', \
                       'tau_b', 'D_b', 'alph_b', 'vv_b', 'Pow_core', 'Pow_beam', \
                       'uns_IC', 'uns_FM', 'uns_OFH', 'uns_core', 'lambda_FH']

    df_cb_uns_pre_SC_cut_ = df_cb_uns_pre_SC_[column_list_cb_]
    
    df_cb_uns_pre_SC_cut_[['beta_par_core', 'alph_c']] = np.log10(df_cb_uns_pre_SC_cut_[['beta_par_core', 'alph_c']])
    df_cb_uns_pre_SC_cut_.rename(columns={"beta_par_core": "log_beta_par_core", "alph_c": "log_alph_c"}, inplace = True)
    
    df_cb_uns_post_SC_ = df_cb_uns_pre_SC_cut_.copy()
    df_cb_uns_post_SC_['cluster'] = gmm_cb.predict(df_cb_uns_pre_SC_cut_.values)
    
    df_cb_uns_post_SC_[['log_beta_par_core', 'log_alph_c']] = \
    pow(10., df_cb_uns_post_SC_[['log_beta_par_core', 'log_alph_c']])
    df_cb_uns_post_SC_.rename(columns={"log_beta_par_core": "beta_par_core", "log_alph_c": "alph_c"}, inplace = True)
    
    df_cb_uns_post_SC_['ins_type'] = np.nan
    ins_types_cb_ = ['Oblique FM (B); resonant with Core', 'IC (B); $T_\perp/T_{||} < 1$', 'Oblique FM (B)', \
                    'Oblique Firehose', 'IC (B); $T_\perp/T_{||} > 1$', 'Parallel Firehose', 'IC (B), unstable core', 'IC (C)']
    for cluster, ins_type in zip(np.arange(8), ins_types_cb_):
        df_cb_uns_post_SC_['ins_type'][df_cb_uns_post_SC_['cluster'] == cluster] = ins_type
    
    df_cb_post_SC_ = df_cb_.copy()
    df_cb_post_SC_['ins_type'] = np.nan
    df_cb_post_SC_.loc[df_cb_uns_post_SC_.index, ['ins_type']] = df_cb_uns_post_SC_['ins_type']
    
    return df_cb_post_SC_

### run SAVIC-C - CB 

In [10]:
df_cb_post_SC = SAVIC_C_CB(df_cb_post_SQ)
df_cb_post_SC

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,unstable,group,Pow_core,Pow_beam,kB_angle,ins_type
0,1.0,1.0,1.0,1.0,0.05,0.5,False,NaN,NaN,NaN,NaN,NaN
1,1.5,2.5,0.8,1.0,0.05,0.5,True,3.0,0.193271,0.000000,0.004137,"IC (B), unstable core"
2,0.5,1.0,1.0,3.5,0.10,1.5,True,5.0,0.000000,0.123028,0.003983,IC (B); $T_\perp/T_{||} > 1$
3,0.8,1.1,1.0,1.2,0.05,1.8,True,5.0,0.000000,0.004923,0.001414,IC (B); $T_\perp/T_{||} > 1$
4,0.5,0.7,0.8,0.8,0.01,0.2,False,NaN,NaN,NaN,NaN,NaN
5,0.8,3.1,1.0,3.9,0.10,1.9,True,1.0,0.214777,0.000728,0.000991,"IC (B), unstable core"


## SAVIC - CB

### load SAVIC - CB processing function

In [11]:
def SAVIC_CoreBeam(df_cb_):
    
    df_cb_post_SP_ = SAVIC_P_CB(df_cb_)
    df_cb_post_SQ_ = SAVIC_Q_CB(df_cb_post_SP_)
    df_cb_post_SC_ = SAVIC_C_CB(df_cb_post_SQ_)
    
    return df_cb_post_SC_

### run SAVIC - CB

In [12]:
df_cb_post_SC = SAVIC_CoreBeam(pd.read_hdf('00_SAVIC_Examples/SAVIC_Examples.h5', key = 'SAVIC-P_CB_pre_P'))
df_cb_post_SC

,beta_par_core,alph_c,tau_b,alph_b,D_b,vv_b,unstable,group,Pow_core,Pow_beam,kB_angle,ins_type
0,1.0,1.0,1.0,1.0,0.05,0.5,False,NaN,NaN,NaN,NaN,NaN
1,1.5,2.5,0.8,1.0,0.05,0.5,True,3.0,0.193271,0.000000,0.004137,"IC (B), unstable core"
2,0.5,1.0,1.0,3.5,0.10,1.5,True,5.0,0.000000,0.123028,0.003983,IC (B); $T_\perp/T_{||} > 1$
3,0.8,1.1,1.0,1.2,0.05,1.8,True,5.0,0.000000,0.004923,0.001414,IC (B); $T_\perp/T_{||} > 1$
4,0.5,0.7,0.8,0.8,0.01,0.2,False,NaN,NaN,NaN,NaN,NaN
5,0.8,3.1,1.0,3.9,0.10,1.9,True,1.0,0.214777,0.000728,0.000991,"IC (B), unstable core"
